In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Bladder Cancer
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Search for exact matches or synonyms
matched_dir = None
for subdir in subdirectories:
    if "bladder" in subdir.lower() and "cancer" in subdir.lower():
        matched_dir = subdir
        break

if not matched_dir:
    print(f"No suitable directory found for {trait}.")
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    exit()

print(f"Selected directory: {matched_dir}")

# 2. Get the clinical and genetic data file paths
cohort_dir = os.path.join(tcga_root_dir, matched_dir)
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

print(f"Clinical file: {os.path.basename(clinical_file_path)}")
print(f"Genetic file: {os.path.basename(genetic_file_path)}")

# 3. Load the data files
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# 4. Print clinical data columns for inspection
print("\nClinical data columns:")
print(clinical_df.columns.tolist())

# Print basic information about the datasets
print(f"\nClinical data shape: {clinical_df.shape}")
print(f"Genetic data shape: {genetic_df.shape}")


Selected directory: TCGA_Bladder_Cancer_(BLCA)
Clinical file: TCGA.BLCA.sampleMap_BLCA_clinicalMatrix
Genetic file: TCGA.BLCA.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_BLCA', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_BLCA', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_completion_success_outcome', 'age_at_initial_pathologic_diagnosis', 'age_began_smoking_in_years', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bladder_carcinoma_extracapsular_extension_status', 'cancer_diagnosis_cancer_type_icd9_text_name', 'chemical_exposure_text', 'clinical_T', 'complete_response_observed', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'd

### Step 2: Find Candidate Demographic Features

In [3]:
# Find candidate age columns
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis',
    'age_began_smoking_in_years',
    'days_to_birth'  # This is often used to calculate age
]

# Find candidate gender columns
candidate_gender_cols = [
    'gender'
]

# Extract the candidate columns from clinical data
# First, we need to load the clinical data
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Bladder_Cancer_(BLCA)")
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_table(clinical_file_path, index_col=0)

# Extract and preview age columns
age_data = clinical_df[candidate_age_cols]
age_preview = preview_df(age_data, n=5)
print("Age columns preview:")
print(age_preview)

# Extract and preview gender columns
gender_data = clinical_df[candidate_gender_cols]
gender_preview = preview_df(gender_data, n=5)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [63, 66, 69, 59, 83], 'age_began_smoking_in_years': [20.0, 15.0, nan, nan, 30.0], 'days_to_birth': [-23323.0, -24428.0, -25259.0, -21848.0, -30520.0]}

Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Step 1: Select appropriate columns for age and gender

# For age, we have three candidate columns:
# - 'age_at_initial_pathologic_diagnosis': Contains direct age values
# - 'age_began_smoking_in_years': Contains smoking initiation age (many NaN values)
# - 'days_to_birth': Contains negative values representing days before birth (essentially age in days)

# Choose the most appropriate column for age
age_col = 'age_at_initial_pathologic_diagnosis'  # This column has clear, direct age values without NaNs

# For gender, we only have one candidate column:
# - 'gender': Contains 'MALE' and 'FEMALE' values
gender_col = 'gender'  # This is the only column and contains valid gender information

# Step 2: Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (bladder cancer) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait="Bladder_Cancer", 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col="Bladder_Cancer")
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait="Bladder_Cancer")

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note="Dataset contains TCGA bladder cancer samples with gene expression and clinical information."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Bladder_Cancer/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 426)
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/TCGA.csv
Clinical data shape: (436, 3)
Number of samples in clinical data: 436
Number of samples in genetic data: 426
Number of common samples: 426
Linked data shape: (426, 19851)


Data shape after handling missing values: (426, 19851)
For the feature 'Bladder_Cancer', the least common label is '0' with 19 occurrences. This represents 4.46% of the dataset.
The distribution of the feature 'Bladder_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 69.0
  75%: 76.0
Min: 34
Max: 90
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 115 occurrences. This represents 27.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bladder_Cancer/TCGA.csv
Preprocessing completed.
